Task 1 and Task 2

In [108]:
# importing required libraries
import numpy as np

# defining a class for inverse kinematics
class inv_kinematics():
  #__init__ function of the class
  # it requires 5 inputs: 3 cartesian coordinates
  # and 2 link lengths, which are the characteristics
  # of the manipulator 
  def __init__(self, x, y, z, d1, d2):
    super().__init__()
    self.x = x
    self.y = y
    self.z = z
    self.d1 = d1
    self.d2 = d2

  # separate function for the stanford manipulator
  def stanford(self):
    # using formulae from the textbook
    theta1 = np.arctan(self.y/self.x)
    r = np.sqrt(self.x**2+self.y**2)
    s = self.z - self.d1
    theta2 = np.arctan(s/r)
    d3 = np.sqrt(r**2+s**2) - self.d2
    # converting from radians to degrees
    theta1 = theta1*180/np.pi
    theta2 = theta2*180/np.pi
    print(theta1, theta2, d3)
  # separate function for the SCARA manipulator
  def scara(self):
    # using formulae from the textbook
    r = abs((self.x**2+self.y**2-self.d1**2-self.d2**2)/(2*self.d1*self.d2))
    theta2 = np.arctan(np.sqrt(abs(1-r**2))/r)
    theta1 = np.arctan(self.y/self.x) - np.arctan((self.d2*np.sin(theta2))/(self.d1+self.d2*np.cos(theta2)))
    d3 = -self.z
    # converting from radians to degrees
    theta1 = theta1*180/np.pi
    theta2 = theta2*180/np.pi
    print(theta1, theta2, d3)

#initialising the inv_kinematics class
inverse = inv_kinematics(0.01,2,1,1,1)
print("Joint parameters of Stanford Manipulator:")
# calling the stanford method
inverse.stanford()
print("Joint parameters of SCARA Manipulator:")
# reinitialising with different values
inverse = inv_kinematics(0.01,2,-1,1,1)
# calling the scara method
inverse.scara()

Joint parameters of Stanford Manipulator:
89.71352348972293 0.0 1.000024999843752
Joint parameters of SCARA Manipulator:
89.42706488225137 0.5729172149431336 1


Task 3

In [109]:
# importing the required libraries
import numpy as np
import sys
import sympy as sym

# using the DH parameters and Jacobian matrix
# code from Assignment 3 
class manipulator():

  def __init__(self, no_of_links, type_of_joints, dh_params):
    super().__init__()
    self.no_of_links = no_of_links
    self.type_of_joints = list(type_of_joints)
    self.dh_params = dh_params
    if len(self.dh_params) != self.no_of_links:
      print("ERROR: Please make sure the number of rows in the DH parameters provided is equal to the number of links.")
      sys.exit()
    self.dh_matrices = []
    for i in range(0, self.no_of_links):    
      temp = self.dh_params[i]
      theta = temp[0]
      d = temp[1]
      a = temp[2]
      alpha = temp[3]
      A1 = np.array([[np.cos(theta), -np.sin(theta), 0, 0],[np.sin(theta), np.cos(theta), 0, 0],[0, 0, 1, 0],[0, 0, 0, 1]])
      A2 = np.array([[1, 0, 0, 0],[0, 1, 0, 0],[0, 0, 1, d],[0, 0, 0, 1]])
      A3 = np.array([[1, 0, 0, a],[0, 1, 0, 0],[0, 0, 1, 0],[0, 0, 0, 1]])
      A4 = np.array([[1, 0, 0, 0],[0, np.cos(alpha), -np.sin(alpha), 0],[0, np.sin(alpha), np.cos(alpha), 0],[0, 0, 0, 1]])
      A_final = A1@A2@A3@A4
      self.dh_matrices.append(A_final)
    self.dh_matrices = np.array(self.dh_matrices)

    self.transform_matrices = []
    self.rotation_matrices = []

    for i in range(1, self.no_of_links+1):
      temp = self.dh_matrices[0]
      for j in range(1, i):
        temp = temp@self.dh_matrices[j]
      rot_temp = temp
      rot_temp = np.delete(rot_temp, (3), axis = 0)
      rot_temp = np.delete(rot_temp, (3), axis = 1)
      self.transform_matrices.append(temp)
      self.rotation_matrices.append(rot_temp)

    self.transform_matrices = np.array(self.transform_matrices)
    self.rotation_matrices = np.array(self.rotation_matrices)

    self.end_effector_posn = np.delete(self.transform_matrices[len(self.transform_matrices)-1]@np.array([[0],[0],[0],[1]]), (3), axis = 0)

  def getJv(self):
    self.z = []
    self.origins = []
    self.z.append(np.array([[0],[0],[1]]))
    self.origins.append(np.array([[0],[0],[0]]))

    for i in range(0, self.no_of_links):
      temp_z = self.rotation_matrices[i]@np.array([[0],[0],[1]])
      self.z.append(temp_z)
      temp_origin = self.transform_matrices[i]@np.array([[0],[0],[0],[1]])
      temp_origin = np.delete(temp_origin, (3), axis = 0)
      self.origins.append(temp_origin)
    
    self.z = np.array(self.z)
    self.origins = np.array(self.origins)  
    self.jacobian = []
    for i in range(0,self.no_of_links):
      if self.type_of_joints[i] == "R":
        temp_1 = np.cross(self.z[i], self.end_effector_posn-self.origins[i], axis = 0)
        temp_2 = self.z[i]
      if self.type_of_joints[i] == "P":
        temp_1 = self.z[i]
        temp_2 = np.array([[0],[0],[0]])
      temp_jacobian = np.vstack((temp_1, temp_2))
      self.jacobian.append(temp_jacobian)
    self.jacobian = np.array(self.jacobian)
    self.jacobian = self.jacobian.reshape((self.no_of_links,6))
    self.jacobian = np.transpose(self.jacobian)
    self.Jv = np.delete(self.jacobian,(3,4,5), axis = 0)
  # creating a new method to calculate the joint
  # velocities, given the Jacobian matrix and 
  # the cartesian velocity.
  # Velocity in x,y and z direction is required
  # as input.
  def joint_velocities(self, xdot, ydot, zdot):
    Xdot =  np.array([[xdot],[ydot],[zdot]])
    # First, it is required to check whether this 
    # velocity is possible in this configuration.
    # This is done by comparing the rank of the
    # Jacobian and the rank of the augmented matrix
    # of the Jacobian and Xdot. If the two are equal,
    # then the velocity is possible.
    rank_J = np.linalg.matrix_rank(self.Jv)
    rank_augmented = np.linalg.matrix_rank(np.hstack((self.Jv,Xdot)))
    if rank_J != rank_augmented:
      print("ERROR: Please make sure the velocities provided by you are possible in this configuration.")
      sys.exit()
    # If the given cartesian velocity is possible,
    # the joint velocities can be calculated by
    # multiplying Xdot with the pseudo-inverse
    # of the Jacobian matrix.
    qdot = np.linalg.pinv(self.Jv)@Xdot

    return qdot

# initialising the manipulator method
stanford = manipulator(3, "RRP", [[0,0,0,-np.pi/2],[0,2,0,np.pi/2],[0,2,0,0]])
jacobian_velocity = stanford.getJv()
qdot = stanford.joint_velocities(10,0,0)
print(qdot)

[[-2.5]
 [ 2.5]
 [ 0. ]]


Task 6

In [112]:
# importing required libraries
import numpy as np
import math

class inv_kinematics():
  # taking a rotation matrix as input
  def __init__(self, R):
    super().__init__()
    self.R  = R
  
  def wrist(self):
    # using the formulae from the textbook
    theta = math.atan2(np.sqrt(abs(1-self.R[2][2]**2)), self.R[2][2])
    phi = math.atan2(self.R[1][2], self.R[0][2])
    psi = math.atan2(-self.R[2][0], self.R[2][1])
    # converting from radians to degrees
    print(theta*180/np.pi, phi*180/np.pi, psi*180/np.pi )

robot = inv_kinematics([[1,0,0],[0,1,0],[0,0,1]])
robot.wrist()

0.0 0.0 0.0


Task 7

In [111]:
# importing required libraries
import numpy as np
import sys
import sympy as sym

# using the DH parameters and Jacobian code
# from Assignment 3
class manipulator():

  def __init__(self, no_of_links, type_of_joints, dh_params):
    super().__init__()
    self.no_of_links = no_of_links
    self.type_of_joints = list(type_of_joints)
    self.dh_params = dh_params
    if len(self.dh_params) != self.no_of_links:
      print("ERROR: Please make sure the number of rows in the DH parameters provided is equal to the number of links.")
      sys.exit()
    self.dh_matrices = []
    for i in range(0, self.no_of_links):    
      temp = self.dh_params[i]
      theta = temp[0]
      d = temp[1]
      a = temp[2]
      alpha = temp[3]
      A1 = np.array([[np.cos(theta), -np.sin(theta), 0, 0],[np.sin(theta), np.cos(theta), 0, 0],[0, 0, 1, 0],[0, 0, 0, 1]])
      A2 = np.array([[1, 0, 0, 0],[0, 1, 0, 0],[0, 0, 1, d],[0, 0, 0, 1]])
      A3 = np.array([[1, 0, 0, a],[0, 1, 0, 0],[0, 0, 1, 0],[0, 0, 0, 1]])
      A4 = np.array([[1, 0, 0, 0],[0, np.cos(alpha), -np.sin(alpha), 0],[0, np.sin(alpha), np.cos(alpha), 0],[0, 0, 0, 1]])
      A_final = A1@A2@A3@A4
      self.dh_matrices.append(A_final)
    self.dh_matrices = np.array(self.dh_matrices)

    self.transform_matrices = []
    self.rotation_matrices = []

    for i in range(1, self.no_of_links+1):
      temp = self.dh_matrices[0]
      for j in range(1, i):
        temp = temp@self.dh_matrices[j]
      rot_temp = temp
      rot_temp = np.delete(rot_temp, (3), axis = 0)
      rot_temp = np.delete(rot_temp, (3), axis = 1)
      self.transform_matrices.append(temp)
      self.rotation_matrices.append(rot_temp)

    self.transform_matrices = np.array(self.transform_matrices)
    self.rotation_matrices = np.array(self.rotation_matrices)

    self.end_effector_posn = np.delete(self.transform_matrices[len(self.transform_matrices)-1]@np.array([[0],[0],[0],[1]]), (3), axis = 0)

    self.z = []
    self.origins = []
    self.z.append(np.array([[0],[0],[1]]))
    self.origins.append(np.array([[0],[0],[0]]))

    for i in range(0, self.no_of_links):
      temp_z = self.rotation_matrices[i]@np.array([[0],[0],[1]])
      self.z.append(temp_z)
      temp_origin = self.transform_matrices[i]@np.array([[0],[0],[0],[1]])
      temp_origin = np.delete(temp_origin, (3), axis = 0)
      self.origins.append(temp_origin)
    
    self.z = np.array(self.z)
    self.origins = np.array(self.origins)  
    self.jacobian = []
    for i in range(0,self.no_of_links):
      if self.type_of_joints[i] == "R":
        temp_1 = np.cross(self.z[i], self.end_effector_posn-self.origins[i], axis = 0)
        temp_2 = self.z[i]
      if self.type_of_joints[i] == "P":
        temp_1 = self.z[i]
        temp_2 = np.array([[0],[0],[0]])
      temp_jacobian = np.vstack((temp_1, temp_2))
      self.jacobian.append(temp_jacobian)
    self.jacobian = np.array(self.jacobian)
    self.jacobian = self.jacobian.reshape((self.no_of_links,6))
    self.jacobian = np.transpose(self.jacobian)
    self.Jv = np.delete(self.jacobian,(3,4,5), axis = 0)

    print("End Effector Position:")
    print(self.end_effector_posn)
    print("Manipulator Jacobian:")
    print(self.jacobian)
    print("Velocity Jacobian (can be multiplied with qdot to obtain end-effector velocity):")
    print(self.Jv)

  # creating a new method to take the joint
  # velocities as input and give the 
  # end-effector cartesian velocities 
  # as output.
  def end_effector_vel(self, qdot):
    qdot = np.array(qdot)
    qdot = np.transpose(qdot)
    end_effector_vel = self.Jv@qdot

    print(end_effector_vel)

print("3D Printer:")
PPP = manipulator(3, "PPP", [[0,1,0,-np.pi/2],[-np.pi/2,2,0,np.pi/2],[0,3,0,0]])
print("End Effector Velocity:")
PPP.end_effector_vel([[1,2,3]])

3D Printer:
End Effector Position:
[[-3.]
 [ 2.]
 [ 1.]]
Manipulator Jacobian:
[[ 0.000000e+00  0.000000e+00 -1.000000e+00]
 [ 0.000000e+00  1.000000e+00  6.123234e-17]
 [ 1.000000e+00  6.123234e-17  6.123234e-17]
 [ 0.000000e+00  0.000000e+00  0.000000e+00]
 [ 0.000000e+00  0.000000e+00  0.000000e+00]
 [ 0.000000e+00  0.000000e+00  0.000000e+00]]
Velocity Jacobian (can be multiplied with qdot to obtain end-effector velocity):
[[ 0.000000e+00  0.000000e+00 -1.000000e+00]
 [ 0.000000e+00  1.000000e+00  6.123234e-17]
 [ 1.000000e+00  6.123234e-17  6.123234e-17]]
End Effector Velocity:
[[-3.]
 [ 2.]
 [ 1.]]
